In [1]:
import pandas as pd
import numpy as np
import os.path
from arabert.arabert.preprocess import ArabertPreprocessor
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix, precision_score , recall_score


In [82]:
#from transformers import BertTokenizer, TFBertModel

In [2]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, BertTokenizer,AutoModel
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures

In [3]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.utils import resample
import logging
import torch
import csv
import optuna

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [6]:
class Dataset:
    def __init__(
        self,
        name,
        train,
        test,
        label_list,
    ):
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list

In [48]:
path=r"C:\Users\96654\zatca_tweets_zakaty_8_1.csv"
df = pd.read_csv(path,header=0)
df

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,Search_word,number,number2,number3
0,snscrape.modules.twitter.Tweet,https://twitter.com/Zatca_care/status/15538843...,2022-07-31 23:24:31+00:00,@mosafr20301 وعليكم السلام\nعزيزي مسفر، تتحمل ...,@mosafr20301 وعليكم السلام\nعزيزي مسفر، تتحمل ...,1553884340077084673,"{'_type': 'snscrape.modules.twitter.User', 'us...",2,0,0,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,ضريبة التصرفات العقارية,[],[],[]
1,snscrape.modules.twitter.Tweet,https://twitter.com/Ali0smile/status/155385782...,2022-07-31 21:39:09+00:00,@f37231843 @sss08423144 @SaudiNews50 اللهم زد ...,@f37231843 @sss08423144 @SaudiNews50 اللهم زد ...,1553857826027610117,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,1,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,ضريبة التصرفات العقارية,[],[],[]
2,snscrape.modules.twitter.Tweet,https://twitter.com/Zatca_care/status/15537899...,2022-07-31 17:09:37+00:00,@Abdullah_F990 مرحباً بك\nعزيزي عبدالله، تتحمل...,@Abdullah_F990 مرحباً بك\nعزيزي عبدالله، تتحمل...,1553789995722440707,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,0,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,ضريبة التصرفات العقارية,[],[],[]
3,snscrape.modules.twitter.Tweet,https://twitter.com/dossaad/status/15536936127...,2022-07-31 10:46:38+00:00,@TQ2030TQ @Mo_Alsuwayed دفع الرسوم لن تحل المش...,@TQ2030TQ @Mo_Alsuwayed دفع الرسوم لن تحل المش...,1553693612709482503,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,0,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,ضريبة التصرفات العقارية,[],[],[]
4,snscrape.modules.twitter.Tweet,https://twitter.com/gou_vip/status/15536686146...,2022-07-31 09:07:18+00:00,@Ask_RD1 الصك \nاحضار الشيك من قبل المشتري \nت...,@Ask_RD1 الصك \nاحضار الشيك من قبل المشتري \nت...,1553668614682730496,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,3,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,ضريبة التصرفات العقارية,[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16736,snscrape.modules.twitter.Tweet,https://twitter.com/OmneyaA03502001/status/153...,2022-06-04 21:58:59+00:00,كل واحد فينا عنده حاجة مميزة = دافع قصادها تمن...,كل واحد فينا عنده حاجة مميزة = دافع قصادها تمن...,1533206710097219585,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,حد الضريبة,[],[],[]
16737,snscrape.modules.twitter.Tweet,https://twitter.com/SaraAlmeldeen1/status/1532...,2022-06-03 21:15:55+00:00,@Omarmhmd155 لو عجزت انك تلاقي حد واحد في وسط ...,@Omarmhmd155 لو عجزت انك تلاقي حد واحد في وسط ...,1532833482061320192,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,2,...,"{'_type': 'snscrape.modules.twitter.User', 'us...","[{'_type': 'snscrape.modules.twitter.User', 'u...",NaN,NaN,NaN,NaN,حد الضريبة,[],[],[]
16738,snscrape.modules.twitter.Tweet,https://twitter.com/Arahmed96/status/153229865...,2022-06-02 09:50:43+00:00,الضريبة الاكبر للموضوع هي انك هتاخد وقت طويل ع...,الضريبة الاكبر للموضوع هي انك هتاخد وقت طويل ع...,1532298658661662721,"{'_type': 'snscrape.modules.twitter.User', 'us...",1,0,3,...,"{'_type': 'snscrape.modules.twitter.User', 'us...",NaN,NaN,NaN,NaN,NaN,حد الضريبة,[],[],[]
16739,snscrape.modules.twitter.Tweet,https://twitter.com/Marysha0/status/1532107347...,2022-06-01 21:10:31+00:00,الشخص ذاته إجى كدرس أختم فيه سنتي ، انو احنا م...,الشخص ذاته إجى كدرس أختم فيه سنتي ، انو احنا م...,1532107347778228225,"{'_type': 'snscrape.modules.twitter.User', 'us...",0,0,4,...,"{'_type': 'snscrape.modules.twitter.User', 'us..

In [10]:
len(df)

1723

In [ ]:
path2=r"C:\Users\96654\OneDrive\Desktop\twitter_ZAKAT.xlsx"
df2 = pd.read_excel(path2,header=0)
df=df[['Tweet Text']].drop_duplicates()
len(df2)

In [49]:
len(df)

16741

In [50]:
len(df2)

NameError: name 'df2' is not defined

In [54]:
df=df.append(df2)

C:\Users\96654\AppData\Local\Temp\ipykernel_19132\799868226.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(df2)


In [51]:
len(df)

16741

In [53]:
#path= r"C:\Users\96654\zatca_tweets22.csv"
#path=r"C:\Users\96654\twee3\mentions.xlsx"
#df = pd.read_excel(path,header=0)
df = df.rename(columns={'content': 'text'})
#df['sentiment'] = np.random.randint(0, 3, df.shape[0])
model_name = "bert-base-arabertv02"
arabert_prep = ArabertPreprocessor(model_name=model_name, keep_emojis=False,replace_urls_emails_mentions=True)

DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'
Q_COLUMN= 'question'
df = df[['text']]
#df = df[['text', 'sentiment']]
df.columns = [DATA_COLUMN]
#df.columns = [DATA_COLUMN, LABEL_COLUMN]

df['orginal'] = df[DATA_COLUMN]
df[DATA_COLUMN] = df[DATA_COLUMN].apply(arabert_prep.preprocess)
#df[LABEL_COLUMN] = df[LABEL_COLUMN].apply(lambda x: label_map[x])


test_AJGT = df




dev_df = pd.DataFrame({
    'id':range(len(test_AJGT)),
   # 'label':test_AJGT["label"],
    'text': test_AJGT["text"].replace(r'\n', ' ', regex=True),

})


In [54]:
len(df)

16741

In [55]:
df .columns

Index(['text', 'orginal'], dtype='object')

In [56]:
dev_df

,id,text
0,0,[مستخدم] وعليكم السلام عزيزي مسفر ، تتحمل الدو...
1,1,[مستخدم] [مستخدم] [مستخدم] اللهم زد وبارك لا ت...
2,2,[مستخدم] مرحبا بك عزيزي عبدالله ، تتحمل الدولة...
3,3,[مستخدم] [مستخدم] دفع الرسوم لن تحل المشكلة ، ...
4,4,[مستخدم] الصك احضار الشيك من قبل المشتري تسجيل...
...,...,...
16736,16736,كل واحد فينا عنده حاجة مميزة = دافع قصادها تمن...
16737,16737,[مستخدم] لو عجزت انك تلاقي حد واحد في وسط علي ...
16738,16738,الضريبة الاكبر للموضوع هي انك هتاخد وقت طويل ع...
16739,16739,الشخص ذاته إجى كدرس أختم فيه سنتي ، انو احنا م...


In [57]:
len(test_AJGT)

16741

In [58]:
df.shape

(16741, 2)

In [59]:
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [60]:
class BERTDataset(Dataset):
    def __init__(self, text, model_name, max_len):
      super(BERTDataset).__init__()
      self.text = text
      #self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = BertTokenizer.from_pretrained(model_name,from_pt=True)
      self.max_len = max_len
      #self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask)

In [61]:
model_name="bert-base-arabertv02"
arabert_prep = ArabertPreprocessor(model_name=model_name)

text = "ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري"
arabert_prep.preprocess(text)

'ولن نبالغ إذا قلنا إن هاتف أو كمبيوتر المكتب في زمننا هذا ضروري'

In [21]:
#from transformers import TFBertForMaskedLM

In [22]:
#get 'https://huggingface.co/aubmindlab/bert-base-arabertv02/resolve/main/tf1_model.tar.gz'

In [23]:
#test2 = test2.rename(columns={'content': 'text'})

In [24]:
max_len=64

In [62]:
test_dataset = BERTDataset(dev_df['text'].to_list(),model_name , max_len)

loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file bert-base-arabertv02\config.json
Model config BertConfig {
  "_name_or_path": "bert-base-arabertv02",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 64000
}



In [63]:
len(test_dataset)

16741

In [64]:
import pickle
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
#save_obj(label_map,'label_map')

In [65]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.fp16 = False
training_args.per_device_train_batch_size = 32
training_args.per_device_eval_batch_size = 32
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 5

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
training_args = TrainingArguments("./train")
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 5e-5
training_args.per_device_train_batch_size = 8
training_args.per_device_eval_batch_size = 8
training_args.gradient_accumulation_steps = 2
training_args.training_argsnum_train_epochs=3
training_args.seed=0
training_args.load_best_model_at_end=True




In [66]:
import tensorflow as tf

In [67]:
model=AutoModelForSequenceClassification.from_pretrained(r'C:\Users\96654\twee3\twitter_large')


loading configuration file C:\Users\96654\twee3\twitter_large\config.json
Model config BertConfig {
  "_name_or_path": "C:\\Users\\96654\\twee3\\twitter_large",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 64000
}

loading weights file C:\Users\96654\twee3\twitt

In [68]:
trainer2 = Trainer(model=model,args = training_args)

In [46]:
test_dataset

In [ ]:
result1= trainer2.predict(test_dataset)

***** Running Prediction *****
  Num examples = 16741
  Batch size = 32


result1

pred=result1.predictions

pred

In [ ]:
df_pred = pd.DataFrame(pred)

In [ ]:
df_pred

In [98]:
final_result=[list(label_map.keys())[list(label_map.values()).index(x)] for x in final_pred]

NameError: name 'final_pred' is not defined

In [37]:

final_pred=np.argmax(result1.predictions, axis=1)

#final_result=[list(label_map.keys())[list(label_map.values()).index(x)] for x in final_pred]
true_text=dev_df['text'].to_list()




In [38]:
final_pred

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [39]:
sentimentResults = pd.DataFrame(
    {'text': true_text,
     'sentiment_BERT': final_pred
    })
#sentimentResults.to_excel('BERT_tweets_RESULTS25_II.xlsx')

In [40]:
sentimentResults

,text,sentiment_BERT
0,"[مستخدم] وعليكم السلام عزيزي مسفر ، تتحمل الدولة ضريبة التصرفات العقارية بنسبة 5 % عما لا يزيد عن 1 , 000 , 000 ريال من مبلغ شراء المسكن الأول ويشمل الوحدات السكنية الجاهزة ووحدات البيع على الخارطة ( فقط ) للمواطن ، ولا يسري الأمر الملكي على البناء الذاتي . يهمنا تقييمك : [رابط]",1
1,[مستخدم] [مستخدم] [مستخدم] اللهم زد وبارك لا تحسب ضريبة التصرفات العقارية اخاف من العين,1
2,"[مستخدم] مرحبا بك عزيزي عبدالله ، تتحمل الدولة ضريبة التصرفات العقارية بنسبة 5 % عما لا يزيد عن 1 , 000 , 000 ريال من مبلغ شراء المسكن الأول ويشمل الوحدات السكنية الجاهزة ووحدات البيع على الخارطة ( فقط ) للمواطن ، ولا يسري الأمر الملكي على البناء الذاتي . يهمنا تقييمك : [رابط]",1
3,[مستخدم] [مستخدم] دفع الرسوم لن تحل المشكلة ، وإنما تزيد في المشكلة . لأن التاجر سوف يحاول تأمين ربحه بكل الوسائل . ولا تنسى ضريبة التصرفات العقارية التي سوف تزيد في السعر في كل عملية مبايعة .,0
4,[مستخدم] الصك احضار الشيك من قبل المشتري تسجيل العقار في ضريبة التصرفات العقارية سداد الضريبة . . حضور الطرفين . التواصل مع أقرب موثق لإجراء الإفراغ خلال دقائق . بالتوفيق ونسعد بخدمتكم .,1
5,[مستخدم] المضحك في الموضوع يا انس انهم ينتظرون تاثير قرار الفدرالي اللي يرفع 0 . 75 ٪ كل ستة اشهر - طيب وين الاثر حق ال 6 اشهر الماضيه من اخر رفع ؟ - وتناسو ان العقار تحمل 5 ٪ ضريبه التصرفات العقارية ولم يحدث انهيار في السوق 50 ٪ على مبدئهم اذا كل 0 . 75 ٪ تنزل 20 ٪ الى 30 ٪,0
6,"[مستخدم] وعليكم السلام عزيزي فيصل ، تتحمل الدولة ضريبة التصرفات العقارية بنسبة 5 % عما لا يزيد عن 1 , 000 , 000 ريال من مبلغ شراء المسكن الأول ويشمل الوحدات السكنية الجاهزة ووحدات البيع على الخارطة فقط ) للمواطن",1
7,"[مستخدم] وعليكم السلام عزيزي فيصل ، تتحمل الدولة ضريبة التصرفات العقارية بنسبة 5 % عما لا يزيد عن 1 , 000 , 000 ريال من مبلغ شراء المسكن الأول ويشمل الوحدات السكنية الجاهزة ووحدات البيع على الخارطة فقط ) للمواطن",1
8,نسمع المطالبة بفرض ضرايب اعلى على العقار ومنتجاته ، ويزعم من يطالب بذلك انها ستحد من ارتفاع السعر . سؤال : - برايكم ماهو اثر المطالبة برسوم الأراضي على الاسعار ؟ - ماهو اثر ضريبة التصرفات العقارية على الاسعار - ماهواثر كود البناء على اسعار المنتج العقاري ؟ اتحفونا بآرائكم .,0
9,و عليكم السلام موجود ضريبة التصرفات العقارية غير ضريبة القيمة المضافة من وجهة نظري حتى لو وضعت ضريبة ثالثه لن ينهار العقار المعادله بسيطه زيادة عدد سكان = زيادة الطلب على العقار = ارتفاع سعره [رابط],1


In [41]:
path=r"C:\Users\96654\zatca_tweets_zakaty_8_1.csv"
df2 = pd.read_csv(path,header=0)

In [42]:
final_df

NameError: name 'final_df' is not defined

In [43]:
final_df = pd.merge(df2,sentimentResults,how="inner" ,left_index=True, right_index=True)

In [44]:
df2

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,inReplyToUser,mentionedUsers,coordinates,place,hashtags,cashtags,Search_word,number,number2,number3
0,snscrape.modules.twitter.Tweet,https://twitter.com/Zatca_care/status/1553884340077084673,2022-07-31 23:24:31+00:00,"@mosafr20301 وعليكم السلام\nعزيزي مسفر، تتحمل الدولة ضريبة التصرفات العقارية بنسبة 5% عما لا يزيد عن 1,000,000ريال من مبلغ شراء المسكن الأول ويشمل الوحدات السكنية الجاهزة ووحدات البيع على الخارطة (فقط) للمواطن، ولا يسري الأمر الملكي على البناء الذاتي.\nيهمنا تقييمك:https://t.co/R8yxq27omp","@mosafr20301 وعليكم السلام\nعزيزي مسفر، تتحمل الدولة ضريبة التصرفات العقارية بنسبة 5% عما لا يزيد عن 1,000,000ريال من مبلغ شراء المسكن الأول ويشمل الوحدات السكنية الجاهزة ووحدات البيع على الخارطة (فقط) للمواطن، ولا يسري الأمر الملكي على البناء الذاتي.\nيهمنا تقييمك:bit.ly/3obuYGr",1553884340077084673,"{'_type': 'snscrape.modules.twitter.User', 'username': 'Zatca_care', 'id': 500475603, 'displayname': 'اسأل الزكاة والضريبة والجمارك', 'description': 'في خدمتكم على مدار الساعة للإجابة على الاستفسارات الزكوية والضريبية والجمركية للاتصال 19993', 'rawDescription': 'في خدمتكم على مدار الساعة للإجابة على الاستفسارات الزكوية والضريبية والجمركية للاتصال 19993', 'descriptionUrls': None, 'verified': True, 'created': '2012-02-23T05:18:21+00:00', 'followersCount': 472256, 'friendsCount': 3, 'statusesCount': 36754, 'favouritesCount': 1, 'listedCount': 630, 'mediaCount': 30, 'location': 'Kingdom of Saudi Arabia', 'protected': False, 'linkUrl': 'http://zatca.gov.sa', 'linkTcourl': 'https://t.co/XjlLighNr9', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1535933738793713666/g16MvLGZ_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/500475603/1651917942', 'label': None, 'url': 'https://twitter.com/Zatca_care'}",2,0,0,...,"{'_type': 'snscrape.modules.twitter.User', 'username': 'mosafr20301', 'id': 1160561853732401152, 'displayname': 'mosafr2030', 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'label': None, 'url': 'https://twitter.com/mosafr20301'}","[{'_type': 'snscrape.modules.twitter.User', 'username': 'mosafr20301', 'id': 1160561853732401152, 'displayname': 'mosafr2030', 'description': None, 'rawDescription': None, 'descriptionUrls': None, 'verified': None, 'created': None, 'followersCount': None, 'friendsCount': None, 'statusesCount': None, 'favouritesCount': None, 'listedCount': None, 'mediaCount': None, 'location': None, 'protected': None, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': None, 'profileBannerUrl': None, 'label': None, 'url': 'https://twitter.com/mosafr20301'}]",NaN,NaN,NaN,NaN,ضريبة التصرفات العقارية,[],[],[]
1,snscrape.modules.twitter.Tweet,https://twitter.com/Ali0smile/status/1553857826027610117,2022-07-31 21:39:09+00:00,@f37231843 @sss08423144 @SaudiNews50 اللهم زد وبارك\nلا تحسب ضريبة التصرفات العقارية اخاف من العين,@f37231843 @sss08423144 @SaudiNews50 اللهم زد وبارك\nلا تحسب ضريبة التصرفات العقارية اخاف من العين,1553857826027610117,"{'_type': 'snscrape.modules.twitter.User', 'username': 'Ali0smile', 'id': 794903984280662016, 'displayname': 'علي الخالدي', 'description': '', 'rawDescription': '', 'descriptionUrls': None, 'verified': False, 'created': '2016-11-05T14:07:33+00:00', 'followersCount': 122, 'friendsCount': 347, 'statusesCount': 7771, 'favouritesCount': 874, 'listedCount': 0, 'mediaCount': 486, 'location': 'المملكة العربية السعوديةالعظمى', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1211606883871707137/Emt3uooS_normal.jpg', 'profileBannerUrl': None, 'label': None, 'url': 'https://twitter.com/Ali0smile'}",0,0

In [45]:
len(final_df)

16741

In [46]:
final_df.to_excel('BERT_tweets_All_8_1.xlsx')

In [79]:
final_df

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,place,hashtags,cashtags,Search_word,number,number2,number3,user2,text,sentiment_BERT
0,snscrape.modules.twitter.Tweet,https://twitter.com/Zatca_care/status/1528343303799857152,2022-05-22 11:53:33+00:00,@MP37607833 عزيزي العميل، للتوضيح لك، يستثى من ضريبة التصرفات العقارية التصرف في العقار بصورة قسرية في حالات نزع الملكية للمنفعة العامة أو وضع اليد المؤقت على العقار،كما يمكنك الإطلاع على المادة الثالثة من خلال الرابط التالي: https://t.co/rmFchPHbpZ\nيهمنا تقييمك: https://t.co/R8yxq27omp,@MP37607833 عزيزي العميل، للتوضيح لك، يستثى من ضريبة التصرفات العقارية التصرف في العقار بصورة قسرية في حالات نزع الملكية للمنفعة العامة أو وضع اليد المؤقت على العقار،كما يمكنك الإطلاع على المادة الثالثة من خلال الرابط التالي: bit.ly/3OWOl2N\nيهمنا تقييمك: bit.ly/3obuYGr,1528343303799857152,"{'_type': 'snscrape.modules.twitter.User', 'username': 'Zatca_care', 'id': 500475603, 'displayname': 'اسأل الزكاة والضريبة والجمارك', 'description': 'في خدمتكم على مدار الساعة للإجابة على الاستفسارات الزكوية والضريبية والجمركية للاتصال 19993', 'rawDescription': 'في خدمتكم على مدار الساعة للإجابة على الاستفسارات الزكوية والضريبية والجمركية للاتصال 19993', 'descriptionUrls': None, 'verified': True, 'created': '2012-02-23T05:18:21+00:00', 'followersCount': 461402, 'friendsCount': 3, 'statusesCount': 31725, 'favouritesCount': 1, 'listedCount': 620, 'mediaCount': 28, 'location': 'Kingdom of Saudi Arabia', 'protected': False, 'linkUrl': 'http://zatca.gov.sa', 'linkTcourl': 'https://t.co/XjlLighNr9', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1439853586230480901/7satIx4O_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/500475603/1651917942', 'label': None, 'url': 'https://twitter.com/Zatca_care'}",0,0,0,...,NaN,NaN,NaN,ضريبة التصرفات العقارية,[],[],[],_typeusernameiddisplaynamedescriptionrawDescriptiondescriptionUrlsverifiedcreatedfollowersCountfriendsCountstatusesCountfavouritesCountlistedCountmediaCountlocationprotectedlinkUrllinkTcourlprofileImageUrlprofileBannerUrllabelurl,[مستخدم] عزيزي العميل ، للتوضيح لك ، يستثى من ضريبة التصرفات العقارية التصرف في العقار بصورة قسرية في حالات نزع الملكية للمنفعة العامة أو وضع اليد المؤقت على العقار ، كما يمكنك الإطلاع على المادة الثالثة من خلال الرابط التالي : [رابط] يهمنا تقييمك : [رابط],1
1,snscrape.modules.twitter.Tweet,https://twitter.com/MP37607833/status/1528341638866780160,2022-05-22 11:46:56+00:00,@Zatca_care @s14432022 هل البيع القسري للعقارات عن طريق محاكم التنفيذ مستثنى من دفع ضريبة التصرفات العقارية للمشتري؟,@Zatca_care @s14432022 هل البيع القسري للعقارات عن طريق محاكم التنفيذ مستثنى من دفع ضريبة التصرفات العقارية للمشتري؟,1528341638866780160,"{'_type': 'snscrape.modules.twitter.User', 'username': 'MP37607833', 'id': 1410347866107883522, 'displayname': 'MP', 'description': 'الله البادي ثم مجد بلادي 🇸🇦', 'rawDescription': 'الله البادي ثم مجد بلادي 🇸🇦', 'descriptionUrls': None, 'verified': False, 'created': '2021-06-30T21:22:06+00:00', 'followersCount': 59, 'friendsCount': 869, 'statusesCount': 1544, 'favouritesCount': 200, 'listedCount': 2, 'mediaCount': 237, 'location': '', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1410348158756982784/jZ4q3pQ5_normal.jpg', 'profileBannerUrl': None, 'label': None, 'url': 'https://twitter.com/MP37607833'}",1,0,0,...,NaN,NaN,NaN,ضريبة التصرفات العقارية,[],[],[],_typeusernameiddisplaynamedescriptionrawDescriptiondescriptionUrlsverifiedcreatedfollowersCountfriendsCountstatusesCountfavouritesCountlistedCountmediaCountlocationprotectedlinkUrllinkTcourlprofileImageUrlprofileBannerUrllabelurl,[مستخدم] [مستخدم] هل البيع القسري للعقارات عن طريق محاكم التنفيذ مستثنى من دفع ضريبة التصرفات العقارية للمشتري ؟,1
2,snscrape.modules.twitter.Tweet,https://twitter.com/SaudiHousingCC/status/1528297752114995200,2022-05-22 08:52:33+00:00,@pilot1403 وعليكم السلام ورحمة الله وبركاته، \nبموجب الأمر الملكي تتحمل الد

In [82]:
len(df_pred)

NameError: name 'df_pred' is not defined

In [55]:
len(final_df)

30145

In [40]:
final_pred = pd.merge(final_df,df_pred,how="inner" ,left_index=True, right_index=True)

NameError: name 'df_pred' is not defined

In [57]:
final_pred

,_type,url,date,content,renderedContent,id,user,replyCount,retweetCount,likeCount,...,place,hashtags,cashtags,Search_word,number,text,sentiment_BERT,0,1,2
0,snscrape.modules.twitter.Tweet,https://twitter.com/01Mftc/status/1500687225226412035,2022-03-07 04:18:10+00:00,🔸 الحقوق والالتزامات والعقوبات وإدارة الضريبة على القيمة المضافة\n🔸 نظرة معمقة لأهم القطاعات المشمولة بضريبة القيمة المضافة\n🔸 ضريبة التصرفات العقارية\n__ __ __ __ __ __ __ __ __,🔸 الحقوق والالتزامات والعقوبات وإدارة الضريبة على القيمة المضافة\n🔸 نظرة معمقة لأهم القطاعات المشمولة بضريبة القيمة المضافة\n🔸 ضريبة التصرفات العقارية\n__ __ __ __ __ __ __ __ __,1500687225226412035,"{'_type': 'snscrape.modules.twitter.User', 'username': '01Mftc', 'id': 1696885376, 'displayname': 'MFTC Training Center', 'description': '\u200f\u200fمركز تدريبي رائد في تقديم الدورات المهنية والشهادات الإحترافية معتمد من المؤسسة العامة للتدريب التقني والمهني - للتواصل واتساب wa.me/message/TB24K4…', 'rawDescription': '\u200f\u200fمركز تدريبي رائد في تقديم الدورات المهنية والشهادات الإحترافية معتمد من المؤسسة العامة للتدريب التقني والمهني - للتواصل واتساب https://t.co/jGKUdDVyC2', 'descriptionUrls': [{'text': 'wa.me/message/TB24K4…', 'url': 'http://wa.me/message/TB24K4CJ6U', 'tcourl': 'https://t.co/jGKUdDVyC2', 'indices': [129, 152]}], 'verified': False, 'created': '2013-08-24T16:23:41+00:00', 'followersCount': 8713, 'friendsCount': 1099, 'statusesCount': 6716, 'favouritesCount': 2694, 'listedCount': 20, 'mediaCount': 5612, 'location': '7466 King Fahad Rd, Jeddah', 'protected': False, 'linkUrl': 'http://www.fin.com.sa', 'linkTcourl': 'https://t.co/t8XAUxyyCN', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1267745360186925057/YDY_UB50_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/1696885376/1615097631', 'label': None, 'url': 'https://twitter.com/01Mftc'}",1,0,0,...,NaN,NaN,NaN,ضريبة التصرفات العقارية,NaN,الحقوق والالتزامات والعقوبات وإدارة الضريبة على القيمة المضافة نظرة معمقة لأهم القطاعات المشمولة بضريبة القيمة المضافة ضريبة التصرفات العقارية _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _,1,-1.131861,3.081951,-1.566389
1,snscrape.modules.twitter.Tweet,https://twitter.com/Silentdream1988/status/1500519238532046855,2022-03-06 17:10:39+00:00,@majed_fantasy @a_mojathil الغرامة وربطها بابشر عشان تضمن انتهاء المشكله وارفع ضريبه التصرفات العقاريه ان لزم الامر عشان ننتهي من البيوت المهجوره👍,@majed_fantasy @a_mojathil الغرامة وربطها بابشر عشان تضمن انتهاء المشكله وارفع ضريبه التصرفات العقاريه ان لزم الامر عشان ننتهي من البيوت المهجوره👍,1500519238532046855,"{'_type': 'snscrape.modules.twitter.User', 'username': 'Silentdream1988', 'id': 612781247, 'displayname': 'عبدالرحمن الغريب', 'description': 'مهندس عالمي', 'rawDescription': 'مهندس عالمي', 'descriptionUrls': None, 'verified': False, 'created': '2012-06-19T19:07:00+00:00', 'followersCount': 1005, 'friendsCount': 556, 'statusesCount': 119708, 'favouritesCount': 10763, 'listedCount': 6, 'mediaCount': 800, 'location': 'أرض الاحلام', 'protected': False, 'linkUrl': None, 'linkTcourl': None, 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1452719708436893699/0pKBBA-V_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/612781247/1632947431', 'label': None, 'url': 'https://twitter.com/Silentdream1988'}",0,0,0,...,NaN,NaN,NaN,ضريبة التصرفات العقارية,NaN,[مستخدم] [مستخدم] الغرامة وربطها بابشر عشان تضمن انتهاء المشكله وارفع ضريبه التصرفات العقاريه ان لزم الامر عشان ننتهي من البيوت المهجوره,0,1.461119,0.284869,-1.089966
2,snscrape.modules.twitter.Tweet,https://twitter.com/Zatca_care/status/1500426378000441344,2022-03-06 11:01:39+00:00,@k400969 أما في حال تم توقيع اتفاقية البيع بعد تعديل لائحة ضريبة التصرفات العقارية في هذه الحالة يجب سداد الضريبة في او قبل تاريخ التوثيق لدى كتابة العدل أو الموثق المعتمد.\nيهمنا تقييمك:⁦https://t.co/R8yxq27omp,@k400969 أما في حال تم توقيع اتفاقية البيع بعد تعديل لائحة ضريبة التصرفات العقارية في هذه الحالة يجب سداد الضريبة في او قبل تاريخ التوثيق لدى كتابة العدل أو الموثق المع

In [43]:
final_pred.to_excel('BERT_tweets_RESULTS_5_22.xlsx')

AttributeError: 'numpy.ndarray' object has no attribute 'to_excel'